In [13]:
import pandas as pd
import numpy as np 
import os 
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings('ignore')

In [2]:
rev = pd.read_csv('../madrid/reviews_detailed/comments_short.csv')

In [3]:
rev.head(100)

,listing_id,id,date,reviewer_id,comments,idiomas,comments_short
0,6369,29428,2010-03-14,84790,simon and arturo have the ultimate location in...,en,arturo ultimate location step bus line want vi...
1,6369,31018,2010-03-23,84338,myself and kristy originally planned on stayin...,en,plan stay arturo week plan change arturo open ...
2,6369,34694,2010-04-10,98655,we had a great time at arturo and simon's ! a ...,en,great time arturo cosy apartment wonderful ter...
3,6369,37146,2010-04-21,109871,i very much enjoyed the stay. \r\nit's a wond...,en,enjoy stay wonderful room bath great apartment...
4,6369,38168,2010-04-26,98901,arturo and simon are polite and friendly hosts...,en,polite friendly host provide pleasant convenie...
...,...,...,...,...,...,...,...
95,21853,273473012,2018-06-06,21220489,je recommande le logement de abdel. tout d'abo...,fr,recommander logement abdel trs attentionn voul...
96,21853,277764607,2018-06-17,182841821,muy buen alojamiento!,es,alojamiento
97,21853,290844505,2018-07-15,49900000,"abdel es muy buen anfitrin, est atento y dispo...",es,abdel anfitrin atento disponible cosa necesit ...
98,24805,23330835,2014-11-29,9091591,"during my stay, i enjoyed all around and had a...",en,stay enjoy fun area location apartment fantast...


In [4]:
rev = rev.dropna()

In [14]:
rev.date=pd.to_datetime(rev.date, format='%Y-%m-%d')

In [19]:
sia = SentimentIntensityAnalyzer()

# Obtener el primer comentario de la columna "comments"
primer_comentario = rev["comments_short"].iloc[96]

# Analizar el sentimiento del comentario utilizando VADER
sentimiento = sia.polarity_scores(primer_comentario)

# Imprimir el resultado
print(sentimiento)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [20]:
import os.path

# Dividir los datos en paquetes de 1000 filas
tamano_paquete = 1000
num_paquete = 0
for inicio in range(0, len(rev), tamano_paquete):
    fin = inicio + tamano_paquete
    paquete = rev.iloc[inicio:fin]

    # Analizar el sentimiento de cada comentario en el paquete
    sia = SentimentIntensityAnalyzer()
    paquete['sentimiento'] = paquete['comments'].apply(lambda comentario: sia.polarity_scores(comentario))

    # Guardar el paquete en un archivo separado
    nombre_archivo = f'paquete_{num_paquete}.csv'
    if not os.path.exists(nombre_archivo):
        paquete.to_csv(nombre_archivo, index=False)
    else:
        paquete.to_csv(nombre_archivo, mode='a', header=False, index=False)
    num_paquete += 1

In [21]:
import pandas as pd
import glob

# Ruta donde se encuentran los archivos CSV
path = r'C:/Users/Pedro Vidales/Desktop/Proyecto-Final_/main/paquetes2/*.csv'

# Lee todos los archivos CSV y crea un dataframe para cada uno
all_files = glob.glob(path)
df_list = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df_list.append(df)

# Concatena los dataframes en uno solo
rev_paquetes2 = pd.concat(df_list, axis=0, ignore_index=True)

# Muestra el resultado
rev_paquetes2.head()

,listing_id,id,date,reviewer_id,comments,idiomas,comments_short,sentimiento
0,6369,29428,2010-03-14,84790,simon and arturo have the ultimate location in...,en,arturo ultimate location step bus line want vi...,"{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'comp..."
1,6369,31018,2010-03-23,84338,myself and kristy originally planned on stayin...,en,plan stay arturo week plan change arturo open ...,"{'neg': 0.017, 'neu': 0.609, 'pos': 0.374, 'co..."
2,6369,34694,2010-04-10,98655,we had a great time at arturo and simon's ! a ...,en,great time arturo cosy apartment wonderful ter...,"{'neg': 0.0, 'neu': 0.612, 'pos': 0.388, 'comp..."
3,6369,37146,2010-04-21,109871,i very much enjoyed the stay. \r\nit's a wond...,en,enjoy stay wonderful room bath great apartment...,"{'neg': 0.0, 'neu': 0.636, 'pos': 0.364, 'comp..."
4,6369,38168,2010-04-26,98901,arturo and simon are polite and friendly hosts...,en,polite friendly host provide pleasant convenie...,"{'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'comp..."


In [22]:
import json
def parse_sentimientos(sentimiento):
    return json.loads(sentimiento.replace("'", "\""))

# Convertimos los datos en formato JSON a un diccionario de Python
rev_paquetes2['sentimiento_dict'] = rev_paquetes2['sentimiento'].apply(parse_sentimientos)

In [24]:
# Creamos cuatro columnas nuevas y extraemos los valores correspondientes a cada clave en cada fila
rev_paquetes2['neg'] = rev_paquetes2['sentimiento_dict'].apply(lambda x: x['neg'])
rev_paquetes2['neu'] = rev_paquetes2['sentimiento_dict'].apply(lambda x: x['neu'])
rev_paquetes2['pos'] = rev_paquetes2['sentimiento_dict'].apply(lambda x: x['pos'])
rev_paquetes2['compound'] = rev_paquetes2['sentimiento_dict'].apply(lambda x: x['compound'])


In [26]:
rev = rev_paquetes2.drop(['sentimiento', 'sentimiento_dict'], axis=1)

In [2]:
rev[rev['neg'] > 0.3].head(50)

NameError: name 'rev' is not defined

In [1]:
rev.info(memory_usage='deep')

NameError: name 'rev' is not defined

In [39]:
rev.comments_short[657688]

'waste money water damage mold photo misleading horrible area'

In [38]:
rev.comments[24664]

'die wohnung war sauber. sehr zentral gelegen.\nwie beschrieben.\nempfehlenswert!'